In [3]:
import tkinter as tk
from tkinter import filedialog
import os
from PIL import Image, ImageTk
import tensorflow as tf
import numpy as np

root = tk.Tk()
root.title("Image Validation App")

# Load your trained model
model = tf.keras.models.load_model('C:\\Users\\Prathiksai\\Downloads\\saved')

# Constants
IMAGE_WIDTH, IMAGE_HEIGHT = 150, 150
NUM_CLASSES = 9

# Define category names for invalid images
invalid_folder_names = {
    0: "blurred_image",
    1: "facing_sideway",
    2: "inverted_image",
    3: "small_sized",
    4: "with_spectacles",
    5: "zoomed_image"
    # Add more categories as needed
}

def is_image_valid_with_model(image_path, model):
    img = Image.open(image_path).resize((IMAGE_WIDTH, IMAGE_HEIGHT))
    img = img.convert("RGB")
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    predictions = model.predict(img_array)
    threshold = 0.5  # Adjust the threshold as needed
    is_valid = np.max(predictions) >= threshold
    return is_valid, predictions

def select_folder():
    folder_path = filedialog.askdirectory()
    process_images(folder_path)

def process_images(folder_path):
    image_viewer = tk.Toplevel(root)
    image_viewer.title("Image Validation Result")

    canvas = tk.Canvas(image_viewer)
    canvas.pack(side="left", fill="both", expand=True)

    scrollbar = tk.Scrollbar(image_viewer, command=canvas.yview)
    scrollbar.pack(side="right", fill="y")
    canvas.configure(yscrollcommand=scrollbar.set)

    frame = tk.Frame(canvas)
    canvas.create_window((0, 0), window=frame, anchor="nw")

    for index, filename in enumerate(os.listdir(folder_path)):
        image_path = os.path.join(folder_path, filename)
        if os.path.isfile(image_path):
            is_valid, predictions = is_image_valid_with_model(image_path, model)
            image = Image.open(image_path)
            image = image.resize((150, 150))
            photo = ImageTk.PhotoImage(image=image)

            image_label = tk.Label(frame, image=photo)
            image_label.photo = photo
            image_label.grid(row=index, column=0, padx=10, pady=10, sticky='w')

            name_label = tk.Label(frame, text=filename)
            name_label.grid(row=index, column=1, padx=10, pady=10, sticky='w')

            if is_valid:
                validation_label = tk.Label(frame, text="Valid")
            else:
                predicted_class_index = np.argmax(predictions)
                category_name = invalid_folder_names.get(predicted_class_index, "Unknown Invalid Category")
                validation_label = tk.Label(frame, text=f"Invalid ({category_name})")

            validation_label.grid(row=index, column=2, padx=10, pady=10, sticky='w')

    def on_configure(event):
        canvas.configure(scrollregion=canvas.bbox("all"))

    frame.bind("<Configure>", on_configure)

    canvas.update_idletasks()
    canvas.configure(scrollregion=canvas.bbox("all"))

    image_viewer.mainloop()

select_button = tk.Button(root, text="Select Folder", command=select_folder)
select_button.pack()

result_label = tk.Label(root, text="")
result_label.pack()

root.mainloop()


1/1 [==============================] - 0s 79ms/step
